<a href="https://colab.research.google.com/github/suubkiim/Pytorch-practice/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module): #nn.Module은 모든 Neural Network Model의 Base Class이다. 
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            #Explain > nn.Conv2d(3, 32, 3, padding=1)
            #첫번째 parameter 인 3은 input_channel_size가 되겠습니다. 
            #두번째 parameter 인 32는 output_volume_size입니다. 
            #즉, conv1 layer를 거쳐 몇장의 필터를 만들어 내고 싶은가? 입니다. 32장의 필터를 만들어 내고 싶으므로, 32가 되겠습니다.
            #세번째 parameter는 kernel_size입니다. Filter_size라고도 불리기도 하는데, 말 그대로 filter의 사이즈를 정의하는 것입니다.
            # 3*3 filter를 사용하고 싶기 때문에 3을 기입했습니다.
            nn.BatchNorm2d(16), # 전 layer의 output channel 수 = filter 수 =16
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))  #16*14*14
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))   #32*7*7
        self.fc = nn.Linear(7*7*32, num_classes)   
        
    def forward(self, x):
        out = self.layer1(x)
        # [100, 16, 14, 14]
        out = self.layer2(out)
        # [100, 32, 7, 7]
        out = out.reshape(out.size(0), -1)
        # out.shape = [100,1568]
        out = self.fc(out)
        #[100,10]
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

summary(model, (1,28,28)) #input size 1 channel, 28*28

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

0it [00:00, ?it/s]

9920512it [00:01, 8723437.04it/s]                            


Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 135711.35it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


1654784it [00:00, 2231674.11it/s]                            
0it [00:00, ?it/s]

Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


8192it [00:00, 51673.83it/s]            


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!
Epoch [1/5], Step [100/600], Loss: 0.1663
Epoch [1/5], Step [200/600], Loss: 0.1263
Epoch [1/5], Step [300/600], Loss: 0.0565
Epoch [1/5], Step [400/600], Loss: 0.1030
Epoch [1/5], Step [500/600], Loss: 0.0421
Epoch [1/5], Step [600/600], Loss: 0.0393
Epoch [2/5], Step [100/600], Loss: 0.0297
Epoch [2/5], Step [200/600], Loss: 0.0693
Epoch [2/5], Step [300/600], Loss: 0.0271
Epoch [2/5], Step [400/600], Loss: 0.0558
Epoch [2/5], Step [500/600], Loss: 0.0461
Epoch [2/5], Step [600/600], Loss: 0.0380
Epoch [3/5], Step [100/600], Loss: 0.0150
Epoch [3/5], Step [200/600], Loss: 0.0244
Epoch [3/5], Step [300/600], Loss: 0.0432
Epoch [3/5], Step [400/600], Loss: 0.0617
Epoch [3/5], Step [500/600], Loss: 0.0600
Epoch [3/5], Step [600/600], Loss: 0.0073
Epoch [4/5], Step [100/600], Loss: 0.0685
Epoch [4/5], Step [200/600], Loss: 0.0064
Epoch [4/5], Step [300/600], Loss: 0.0492
Epoch [4/5], Ste

https://greeksharifa.github.io/pytorch/2018/11/02/pytorch-usage-02-Linear-Regression-Model/#import

* from torch import nn: nn은 Neural Network의 약자이다. torch의 nn 라이브러리는 Neural Network의 모든 것을 포괄하며, Deep-Learning의 가장 기본이 되는 1-Layer Linear Model도 nn.Linear 클래스를 사용한다. 이 예제에서도 nn.Linear를 쓴다.
> * nn.Module은 모든 Neural Network Model의 Base Class이다. 모든 Neural Network Model(흔히 Net이라고 쓴다)은 nn.Module의 subclass이다. nn.Module을 상속한 어떤 subclass가 Neural Network Model로 사용되려면 다음 두 메서드를 override해야 한다.
>> __init__(self): Initialize. 여러분이 사용하고 싶은, Model에 사용될 구성 요소들을 정의 및 초기화한다. 대개 다음과 같이 사용된다.
>>> * self.conv1 = nn.Conv2d(1, 20, 5)
>>> * self.conv2 = nn.Conv2d(20, 20, 5)
>>> * self.linear1 = nn.Linear(1, 20, bias=True)
>> forward(self, x): Specify the connections. __init__에서 정의된 요소들을 잘 연결하여 모델을 구성한다. Nested Tree Structure가 될 수도 있다. 주로 다음처럼 사용된다.
>>> * x = F.relu(self.conv1(x))
>>> * return F.relu(self.conv2(x))

> * 다른 말로는 위의 두 메서드를 override하기만 하면 손쉽게 Custom net을 구현할 수 있다는 뜻이기도 하다.